<a href="https://colab.research.google.com/github/ramnayudu/Telememory_POC/blob/main/Telememory_POC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Telememory POC 

**POC Requirements**

1. Reading Frames from a video - **Using OpenCV and Decord**
2. Finding all Faces in the Frame - **Using Fave_Recognition**
3. Clustering the faces into groups by using the distance between facial embeddings - **Feature Extraction using Deep Metric learning with the help of Face_Recogintion built using state of the art face recognition built with  deep learing**
4.The next step is **Face Recognition** is to compute the vectors for each image using the same network we used above and then compare the embeddings with the rest of the embeddings we have **(Not part of POC but can be achive using same Face_Recognition)**



**Step1 : Reading Frames From a video (Upload a video to content folder)**

In [ ]:
pip install decord

In [ ]:
import cv2  # still used to save images out
import os
import numpy as np
from decord import VideoReader
from decord import cpu, gpu


def get_frames(video_path, frames_dir, overwrite=False, start=-1, end=-1, every=1):
    """
    Extract frames from a video using decord's VideoReader
    :param video_path: path of the video
    :param frames_dir: the directory to save the frames
    :param overwrite: to overwrite frames that already exist?
    :param start: start frame
    :param end: end frame
    :param every: frame spacing
    :return: count of images saved
    """

    video_path = os.path.normpath(video_path)  # make the paths OS (Windows) compatible
    frames_dir = os.path.normpath(frames_dir)  # make the paths OS (Windows) compatible

    video_dir, video_filename = os.path.split(video_path)  # get the video path and filename from the path

    assert os.path.exists(video_path)  # assert the video file exists

    # load the VideoReader
    vr = VideoReader(video_path, ctx=cpu(0))  # can set to cpu or gpu .. ctx=gpu(0)
                     
    if start < 0:  # if start isn't specified lets assume 0
        start = 0
    if end < 0:  # if end isn't specified assume the end of the video
        end = len(vr)

    frames_list = list(range(start, end, every))
    saved_count = 0

    if every > 25 and len(frames_list) < 1000:  # this is faster for every > 25 frames and can fit in memory
        frames = vr.get_batch(frames_list).asnumpy()
        print("frames")

        for index, frame in zip(frames_list, frames):  # lets loop through the frames until the end
            save_path = os.path.join(frames_dir, video_filename, "{:010d}.jpg".format(index))  # create the save path
            if not os.path.exists(save_path) or overwrite:  # if it doesn't exist or we want to overwrite anyways
                cv2.imwrite(save_path, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))  # save the extracted image
                saved_count += 1  # increment our counter by one

    else:  # this is faster for every <25 and consumes small memory
        for index in range(start, end):  # lets loop through the frames until the end
            frame = vr[index]  # read an image from the capture
            
            if index % every == 0:  # if this is a frame we want to write out based on the 'every' argument
                save_path = os.path.join(frames_dir, video_filename, "{:010d}.jpg".format(index))  # create the save path
                if not os.path.exists(save_path) or overwrite:  # if it doesn't exist or we want to overwrite anyways
                    cv2.imwrite(save_path, cv2.cvtColor(frame.asnumpy(), cv2.COLOR_RGB2BGR))  # save the extracted image
                    saved_count += 1  # increment our counter by one

    return saved_count  # and return the count of the images we saved


def frames_from_video(video_path, frames_dir, overwrite=False, every=1):
    """
    Extracts the frames from a video
    :param video_path: path to the video
    :param frames_dir: directory to save the frames
    :param overwrite: overwrite frames if they exist?
    :param every: extract every this many frames
    :return: path to the directory where the frames were saved, or None if fails
    """

    video_path = os.path.normpath(video_path)  # make the paths OS (Windows) compatible
    frames_dir = os.path.normpath(frames_dir)  # make the paths OS (Windows) compatible

    video_dir, video_filename = os.path.split(video_path)  # get the video path and filename from the path

    # make directory to save frames, its a sub dir in the frames_dir with the video name
    os.makedirs(os.path.join(frames_dir, video_filename), exist_ok=True)
    
    print("Extracting frames from {}".format(video_filename))
    
    get_frames(video_path, frames_dir, every=every)  # let's now extract the frames

    return os.path.join(frames_dir, video_filename)  # when done return the directory containing the frames


if __name__ == '__main__':
    # test it
    frames_from_video(video_path='sample.mp4', frames_dir='test_frames', overwrite=False, every=5)

**Nowadays computers will in general have various CPU cores that can get things done in parallel. So we can extend out upon the above code to add parallel processing over all CPU cores.**


**Step 2 : Face Detection and Feature Extraction using Deep metric learning (Vectors or facial embeddings)**

**Step 3 : Change Runtime to GPU and create Images folder and upload any of the images that we got form previous step (we will automate this in actual implementation)**

In [ ]:
pip install face_recognition

In [ ]:
from imutils import paths
from matplotlib import pyplot as plt
from PIL import Image
import face_recognition
import cv2
import os
 
#get paths of each file in folder named Images
#Images here contains my data(folders of various persons)
imagePaths = list(paths.list_images('Images'))
knownEncodings = []
# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
    # load the input image and convert it from BGR (OpenCV ordering)
    # to dlib ordering (RGB)
    image = cv2.imread(imagePath)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #Use Face_recognition to locate faces
    boxes = face_recognition.face_locations(rgb,model='hog')
    print(boxes)
    top, right, bottom, left = boxes[0]
    face_image = image[top:bottom, left:right]
    plt.imshow(face_image)
    # compute the facial embedding for the face
    encodings = face_recognition.face_encodings(rgb, boxes)
    # loop over the encodings
    for encoding in encodings:
        knownEncodings.append(encoding)
#save emcodings along with their names in dictionary data
data = {"encodings": knownEncodings}
print(data)

**Step 3: Face Recognition by comparing the vectors or facial embeddings (Not part of POC)**

**We can use Google Vertex AI to save , deploy and to inference. i played with Vertex AI for some time and planned to deploy there but a billing account should be linked with the project so couldnt able to do that for POC**